In [3]:
#!pip install torchvision

In [2]:
from aicspylibczi import CziFile
import czifile
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import cv2
import os
import imageio
import ffmpeg
import time
import pandas as pd
from cellpose import io, models
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import json
import glob
import itertools
from PIL import Image
import torch.nn.functional as F
from utils import *
cudnn.benchmark = True
plt.ion()   # interactive mode

In [3]:

class VideoDataMIP:
    def __init__(self, files):
        self.data = {
        }
        
        for category, num in files:
            print(f"Loading in MIP {num}")
            assert category == 'mip', "Can't load non Mip file"
            file = {}
            file['video'] = get_file(category, num)
            
            frames, shp = file['video'].read_image(C=0)
            frames = scale_img(frames.squeeze())
            file['frames'] = frames
            print(f"frames {num}: {frames.shape}")
            file['masks'] = binarize_video(frames)           
    
            self.data[num] = file    
    def extract_all_traces(self, file_num, sequence_length, hist_length=2):
        # hist length is how many frames of history
        frames, masks = self.data[file_num]['frames'], self.data[file_num]['masks']
        N = len(frames)
        s = 0
        all_traces = []
        all_videos = []
        for i in range(N // sequence_length):
            print(f"Extracting traces from {s}:{s+sequence_length}")
            data, videos = extract_traces(frames[s:s+sequence_length], masks[s:s+sequence_length], hist=hist_length)
            s += sequence_length
            all_traces = all_traces + data
            all_videos = all_videos + videos
        
        if(N % sequence_length > 0):
            data, videos = extract_traces(frames[-1*sequence_length:], masks[-1*sequence_length:], hist=hist_length)
            all_traces = all_traces + data
            all_videos = all_videos + videos        
        self.data[file_num]['traces'] = all_traces
        self.data[file_num]['trace_videos'] = all_videos
            

In [3]:
fake_data = [
    {
        "box": [(1, 2, 3, 4), (1, 2, 3, 5)],
        "masks": [np.ones([100, 100]), np.ones([100, 100])],
        "patch": [np.ones([100, 100]), np.ones([100, 100])]
    },
    {
        "box": [(1, 2, 3, 4), (1, 2, 3, 5)],
        "masks": [np.ones([100, 100]), np.ones((100, 100))],
        "patch": [np.ones([100, 100]), np.ones((100, 100))]
    },
    {
        "box": [(1, 2, 3, 4), (1, 2, 3, 5)],
        "masks": [np.ones([100, 100]), np.ones((100, 100))],
        "patch": [np.ones([100, 100]), np.ones((100, 100))]
    },
    {
        "box": [(1, 2, 3, 4), (1, 2, 3, 5)],
        "masks": [np.ones([100, 100]), np.ones((100, 100))],
        "patch": [np.ones([100, 100]), np.ones((100, 100))]
    },
    {
        "box": [(1, 2, 3, 4), (1, 2, 3, 5)],
        "masks": [np.ones([100, 100]), np.ones((100, 100))],
        "patch": [np.ones([100, 100]), np.ones((100, 100))]
    },
    {
        "box": [(1, 2, 3, 4), (1, 2, 3, 5)],
        "masks": [np.ones([100, 100]), np.ones((100, 100))],
        "patch": [np.ones([100, 100]), np.ones((100, 100))]
    },
    {
        "box": [(1, 2, 3, 4), (1, 2, 3, 5)],
        "masks": [np.ones([100, 100]), np.ones((100, 100))],
        "patch": [np.ones([100, 100]), np.ones((100, 100))]
    },
    {
        "box": [(1, 2, 3, 4), (1, 2, 3, 5)],
        "masks": [np.ones([100, 100]), np.ones((100, 100))],
        "patch": [np.ones([100, 100]), np.ones((100, 100))]
    },
    {
        "box": [(1, 2, 3, 4), (1, 2, 3, 5)],
        "masks": [np.ones([100, 100]), np.ones((100, 100))],
        "patch": [np.ones([100, 100]), np.ones((100, 100))]
    },
    {
        "box": [(1, 2, 3, 4), (1, 2, 3, 5)],
        "masks": [np.ones([100, 100]), np.ones((100, 100))],
        "patch": [np.ones([100, 100]), np.ones((100, 100))]
    }
    
]

fake_data[0]["masks"][1].flatten().shape

(10000,)

In [44]:
from skimage.measure import centroid


box_shape = (180, 180) #TO DO: find the biggest box and set it to this
X = 10

class CellBoxMaskPatch(torch.utils.data.Dataset):
    #input will be a Directory name, function is TO DO
    def __init__(
        self,
        files, 
        X=X):
        
        self.video_extractor = VideoDataMIP(files)

        # list of dictionaries, each dictionary is one cell tracked throughout a video
        #each cell has 3 data types in a dictionary
            #mask, box and patches

        for i in files:
            self.video_extractor.extract_all_traces(i[1], X)
        
        
        self.cell_dict = []

        for key in self.video_extractor.data:
            entry = self.video_extractor.data[key]["traces"]
            for cell in entry:
                print(cell.keys())
                self.cell_dict.append({
                    "boxes": cell["boxes"],
                    "patches": cell["patches"],
                    "masks": cell["masks"]
                })
            

        self.num_cells = len(self.cell_dict)
        # print("the number is cells is", self.num_cells)
        # first_cell = self.cell_dict[0]
        # print("this is the keys of the first cell", first_cell.keys())
        # print("this is the amount of boxes in this cell", len(first_cell['boxes']))


        # self.num_timesteps, self.list_timesteps_start, self.list_timesteps_end = self.get_num_timesteps()

    def __len__(self):
        return self.num_cells

    # def get_num_timesteps(self):
    #     #count the number of items in each list of each cell
    #     count= 0
    #     list_counts_start = [0]
    #     list_counts_end = []
    #     for cell in self.cell_dict:
    #         current_count = len(cell['boxes'])
    #         count+=current_count
    #         list_counts_end.append(count)
    #         list_counts_start.append(count)
    #     print("there are ", count, " timesteps of ", self.num_cells, " cells ", "this is what index they all start at \n", list_counts_start, "\n and end \n", list_counts_end)
    #     return count, list_counts_start[:-1], list_counts_end#list counts returns the start of every cell

    # def binary_search(self, box_end, box_starts, number):
    #     #performs a binary search on the cells, to find the right timestep and cell number for a given index
    #     #if this gets slow, can edit to create a dictionary in get_num_timesteps
    #     left, right = 0, len(box_starts) - 1
    #     while left <= right:
    #         mid = left + (right - left) // 2
    #         if box_starts[mid] <= number < box_end[mid]:
    #             # Found the box
    #             box_number = mid  
    #             position_in_box = number - box_starts[mid]
    #             # print (number,  " found in box ", box_number, ", " , position_in_bsox)
    #             return box_number, position_in_box
    #         elif number < box_starts[mid]:
    #             right = mid - 1
    #         else:
    #             left = mid + 1
    
    #     # Number not found in any box
    #     raise ValueError(f"Number {number} is not found in any cell and timestep combination. Length shoudl be at most", self.num_timesteps)
        

    def get_centroids(cell_data):
        boxes, masks = np.array(cell_data['boxes']), cell_data['masks']
        N = len(masks)
        res = []
        centroids = [skm.centroid(binary.astype(np.uint8)) for binary in masks]
        for i in range(N):
            c = centroids[i]
            ymin, xmin = boxes[i][:2]
            res.append([xmin+c[0], ymin+c[1]])
        return(np.array(res) - res[0,:]) 
   
    def pad_arrays(self, array, pad_amt=1000):
        print(pad_amt)
        print("array shape 0 is", array.shape[0])
        print("array shape 1", array.shape[1])

        print("x would be", pad_amt-array.shape[0])
        print("y would be", pad_amt-array.shape[1])
        pad_width = ((0, pad_amt - array.shape[0]), (0, pad_amt - array.shape[1]))
        print(pad_width)

        padded_array = np.pad(array, pad_width, mode='constant')
        return padded_array

    def __getitem__(self, idx):
        #should T dictiaries for . 
        #If any cells don't exist (ie we look for index n, n+1, n+2, n+3..) and the list is of 
        #length n+2, everything that doesn't exist (n+2, n+3...) should just be self.end, our "end token"
    
        #turn start and cell
        # print(self.list_timesteps_start, self.list_timesteps_end)

        # cell_idx, start = self.binary_search(self.list_timesteps_end, self.list_timesteps_start, idx)

        cell = self.cell_dict[idx]

        boxes = cell["boxes"]
        masks = cell["masks"]
        patches = cell["patches"]


        #make the masks all 1's for all the cells
        for cell_mask in np.arange(len(masks)): #there are 67 cells, each cell is a list of 10 cells
            print("cell mask ", cell_mask, " should be a number btw 0 and 67 ")
            for cell_time in masks[cell_mask]:
                cell_time = np.array(cell_time, dtype=np.int32)
                cell_time = np.where(cell_time < 0, cell_time, 1)
                print("cell time shape and cell time", cell_time.shape, cell_time)
                cell_time = self.pad_arrays(cell_time)
                masks[cell_mask] = cell_time


        for cell_patch in np.arange(len(patches)):
            for cell_time in patches[cell_patch]:
                
                cell_time = self.pad_arrays(cell_time)
                masks[cell_mask] = cell_time


        # box = boxes[start]
        # mask = masks[start]
        # patch = patches[start]

        # # Get the dictionaries for the next T cells
        # next_T_frames_box = []
        # next_T_frames_masks = []
        # next_T_frames_patches = []
        # for i in range(1, self.T + 1):
        #     # print("Getting next frame", i)
        #     if start + i < self.list_timesteps_end[cell_idx]-self.list_timesteps_start[cell_idx]:
        #         next_T_frames_box.append(boxes[start + i])
        #         next_T_frames_masks.append(masks[start+1].flatten())
        #         #change all the numbers to smth else --> do do mask
        #         next_T_frames_patches.append(patches[start+i].flatten())
            
        
        # print("next T frames shape", np.array(next_T_frames_box).shape)
        #returns tuples, list of tuples
                # from [1, d1, d2, d3, 10] becomes [B, 10, d]. #centroids [10, 2] ---> [B, 10, 2]

        centroids = ([self.get_centroids(t) for t in cell])

        return np.array(masks, patches),                 centroids

In [18]:
from torch.utils.data import random_split

root_dir = "/mnt/datadisk/"
subdirs = ['FactinProcessed', 'FactinMIP']

mip_video_files = [
    ('mip', 3),
    ('mip', 6),
    ('mip', 9),
]

mip_test_files = [('mip', 3)] #so that I dont keep crashing the kernel

# file = root_dir + subdirs

video_extractor = VideoDataMIP(mip_test_files)

        # list of dictionaries, each dictionary is one cell tracked throughout a video
        #each cell has 3 data types in a dictionary
            #mask, box and patches


Loading in MIP 3
Loading dicty_factin_pip3-03_MIP.czi with dims [{'X': (0, 474), 'Y': (0, 2048), 'C': (0, 2), 'T': (0, 90)}]
frames 3: (90, 2048, 474)


In [23]:
files = [('mip', 3)]

video_extractor = VideoDataMIP(files)

# list of dictionaries, each dictionary is one cell tracked throughout a video
#each cell has 3 data types in a dictionary
#mask, box and patches
X=10

for i in files:
    video_extractor.extract_all_traces(i[1], X)


Loading in MIP 3
Loading dicty_factin_pip3-03_MIP.czi with dims [{'X': (0, 474), 'Y': (0, 2048), 'C': (0, 2), 'T': (0, 90)}]
frames 3: (90, 2048, 474)
Extracting traces from 0:10
Extracting cell  0
Extracting cell  1
Extracting cell  2
Extracting cell  3
Extracting cell  4
Extracting cell  5
Extracting cell  6
Extracting traces from 10:20
Extracting cell  0
Extracting cell  1
Extracting cell  2
Extracting cell  3
Extracting cell  4
Extracting cell  5
Extracting cell  6
Extracting cell  7
Extracting traces from 20:30
Extracting cell  0
Extracting cell  1
Extracting cell  2
Extracting cell  3
Extracting cell  4
Extracting cell  5
Extracting cell  6
Extracting cell  7
Extracting cell  8
Extracting traces from 30:40
Extracting cell  0
Extracting cell  1
Extracting cell  2
Extracting cell  3
Extracting cell  4
Extracting cell  5
Extracting cell  6
Extracting cell  7
Extracting traces from 40:50
Extracting cell  0
Extracting cell  1
Extracting cell  2
Extracting cell  3
Extracting cell  4
Ex

In [70]:
import skimage.measure as skm
        
        
cell_dict = []

for key in video_extractor.data:
            entry = video_extractor.data[key]["traces"]
            for cell in entry:
                # print("cell is of type", type(cell))
                sequence = []
                patches = [np.array(p) for p in cell["patches"]]
                boxes = [np.array(b) for b in cell['boxes']]
                masks = [np.array(m) for m in cell['masks']]
                # print("first patch", patches[0])
                # print("first patch as arraty", np.array(patches[0]))
                # print("first patch shape", np.array(patches[0]).shape)

                # print("first mask", masks[0])
                # print("first mask as arraty", np.array(masks[0]))
                # print("first mask SHAPPEPEEEE", np.array(masks[0]).shape)

                # for i in np.arange(len(patches)):
                # #     sequence.append({
                # #         "boxes": boxes[i],
                # #         "patches": patches[i],
                # #         "masks": masks[i]
                # #     })
                # #     print(sequence)
                #         sequence.append((boxes, patches, masks))
                #         # print("sequence should be ten lists of 3",len(sequence))
                #         # print("here is the first list of ten, should be 3 long", len(sequence[0]))
                cell_dict.append((boxes, masks, patches)) #cell dict is a list of 3 types by sequence

num_cells = len(cell_dict) #this is a list of how many sequences we have


cell_sequences = cell_dict[0]  #this is the first sequence of 10 cells
boxes = cell_sequences[0]
patches = cell_sequences[1]
masks = cell_sequences[2]


def get_centroids(cell_data):
        boxes, masks = np.array(cell_data['boxes']), cell_data['masks']
        N = len(masks)
        res = []
        centroids = [skm.centroid(binary.astype(np.uint8)) for binary in masks]
        for i in range(N):
            c = centroids[i]
            ymin, xmin = boxes[i][:2]
            res.append([xmin+c[0], ymin+c[1]])
        return(np.array(res) - res[0,:]) 
   
def pad_arrays(array, pad_amt=200):
        print(pad_amt)
        print("array shape 0 is", array.shape[0])
        print("array shape 1", array.shape[1])

        print("x would be", pad_amt-array.shape[0])
        print("y would be", pad_amt-array.shape[1])
        pad_width = ((0, pad_amt - array.shape[0]), (0, pad_amt - array.shape[1]))
        print(pad_width)

        padded_array = np.pad(array, pad_width, mode='constant')
        return padded_array


#make the masks all 1's for all the cells
for cell_mask_num in np.arange(len(masks)): #should be sequence length (10) masks
        # print("cell_mask_num", cell_mask_num)
        cell_time = np.array(masks[cell_mask_num], dtype=np.int32)
        # print("this should be one mask", cell_time)
        cell_time = np.where(cell_time >= 0, cell_time, 1)
        # print("min and max", np.min(cell_time), np.max(cell_time))
        print("cell time shape and cell time 1 ", cell_time.shape)
        cell_time = pad_arrays(cell_time)
        print("cell time shape and cell time 2", cell_time.shape)

        masks[cell_mask_num] = cell_time


        cell_time_patch = np.array(masks[cell_mask_num], dtype=np.int32)
        cell_time_patch = pad_arrays(cell_time_patch)

        masks[cell_mask_num] = cell_time_patch

cell time shape and cell time 1  (111, 56)
200
array shape 0 is 111
array shape 1 56
x would be 89
y would be 144
((0, 89), (0, 144))
cell time shape and cell time 2 (200, 200)
200
array shape 0 is 200
array shape 1 200
x would be 0
y would be 0
((0, 0), (0, 0))
cell time shape and cell time 1  (102, 57)
200
array shape 0 is 102
array shape 1 57
x would be 98
y would be 143
((0, 98), (0, 143))
cell time shape and cell time 2 (200, 200)
200
array shape 0 is 200
array shape 1 200
x would be 0
y would be 0
((0, 0), (0, 0))
cell time shape and cell time 1  (83, 55)
200
array shape 0 is 83
array shape 1 55
x would be 117
y would be 145
((0, 117), (0, 145))
cell time shape and cell time 2 (200, 200)
200
array shape 0 is 200
array shape 1 200
x would be 0
y would be 0
((0, 0), (0, 0))
cell time shape and cell time 1  (151, 118)
200
array shape 0 is 151
array shape 1 118
x would be 49
y would be 82
((0, 49), (0, 82))
cell time shape and cell time 2 (200, 200)
200
array shape 0 is 200
array sha

In [59]:
masks = [v['masks'] for v in video_extractor.data[3]['traces']]#this is a list of 10 masks


import numpy as np

def pad_arrays(array, padding_amt=200):
    pad_width = ((0, 200 - array.shape[0]), (0, 200 - array.shape[1]))
    padded_array = np.pad(array, pad_width, mode='constant')
    return padded_array


#get make all the values == 1

#padding
shapes = []
for cell_mask in masks: #there are 67 cells, each cell is a list of 10 cells
    for cell_time in cell_mask:
        cell_time = np.array(cell_time, dtype=np.int32)
        cell_time = np.where(cell_time < 0, cell_time, 1)
        # print(cell_time.shape)
        
        cell_time = pad_arrays(cell_time)
        shapes.append(cell_time.shape)
        #gonna pad to [180, 180], but can be changed after

    
print(np.max(shapes))
print(np.min(shapes))

200
200


In [73]:
cell_time = np.array([[1, 2, 3], [4, 5, 6]])

print(np.max(cell_time))

cell_time = np.where(cell_time < 1, cell_time, 1)


print(np.max(cell_time))



6
1


In [27]:
from skimage.measure import centroid

test_patches = video_extractor.data[3]['traces'][0]['patches'] #this is a list of X paddings

#get centroids

centroids = ([centroid(t) for t in test_patches])
centroids

[array([62.97384554, 26.70922169]),
 array([52.6810371 , 26.99937134]),
 array([45.47522825, 28.61106318]),
 array([85.75728009, 60.66285577]),
 array([32.28763764, 23.48785023]),
 array([30.01306352, 30.7486319 ]),
 array([19.88243374, 16.62626154]),
 array([11.69016281, 12.22928183]),
 array([57.40563179, 32.18842609]),
 array([55.30272335, 28.77540539])]

In [45]:
from torch.utils.data import random_split

mip_video_files = [
    ('mip', 3)
]

dataset = CellBoxMaskPatch(mip_video_files, 10) # file, S, T

train, eval, test = random_split(dataset, [0.4, 0.2, 0.4])

input_datasets = {}
input_datasets["train"] = train
input_datasets["eval"] = eval
input_datasets["test"] = test

Loading in MIP 3
Loading dicty_factin_pip3-03_MIP.czi with dims [{'X': (0, 474), 'Y': (0, 2048), 'C': (0, 2), 'T': (0, 90)}]
frames 3: (90, 2048, 474)
Extracting traces from 0:10
Extracting cell  0
Extracting cell  1
Extracting cell  2
Extracting cell  3
Extracting cell  4
Extracting cell  5
Extracting cell  6
Extracting traces from 10:20
Extracting cell  0
Extracting cell  1
Extracting cell  2
Extracting cell  3
Extracting cell  4
Extracting cell  5
Extracting cell  6
Extracting cell  7
Extracting traces from 20:30
Extracting cell  0
Extracting cell  1
Extracting cell  2
Extracting cell  3
Extracting cell  4
Extracting cell  5
Extracting cell  6
Extracting cell  7
Extracting cell  8
Extracting traces from 30:40
Extracting cell  0
Extracting cell  1
Extracting cell  2
Extracting cell  3
Extracting cell  4
Extracting cell  5
Extracting cell  6
Extracting cell  7
Extracting traces from 40:50
Extracting cell  0
Extracting cell  1
Extracting cell  2
Extracting cell  3
Extracting cell  4
Ex

In [90]:
# timestep = [[0] for t in range(T)]
# timestep

# batch = ((np.array([1 , 2, 3]), 1, 2) , 3), ((np.array([1 , 2, 3]), 1, 2) , 3)
# current_boxes = torch.stack([torch.tensor(b[0][0], dtype=torch.long) for b in batch], dim=0)
# current_boxes
        #current     #next   #boxes                     #masks
batch  = [[1, 2, 3], [[[1,2, 3, 4], [1, 2, 3,4]], [[1, 2, 3], [1, 2, 3]], [[1, 2, 3], [1,2, 3]]]], [[1, 2, 3], [[[1,2, 3, 4], [1, 2, 3,4]], [[1, 2, 3], [1, 2, 3]], [[1, 2, 3], [1,2, 3]]]]



next_boxes = torch.stack([torch.tensor(b[1][0], dtype=torch.long) for b in batch], dim=0)


In [46]:
def collate_fn(batch, mode_box, mode_mask, mode_patch):
    #this should stick the batches together in way that makes sense in a torch stack manner
    #we want to return dim [B, 1, d], [B, T, d] where d is a combination of 4 (for boxes) and the flattened image shape
    #taking only taking one cell from sequence before and T is sequence after
    #if mode_box is true, we want to return [B, 1, 4] and [B, T, 4]
    #if mode_mask is true, we want to return [B, 1, (10,10).flatten] and [B, T, (10,10).flatten]
    #input (cur_box, cur_mask, cur_patch), (centroids)
  
    # current_boxes = torch.stack([torch.tensor(b[0][0], dtype=torch.long) for b in batch], dim=0)
    current_masks = torch.stack([torch.tensor(b[0][0], dtype=torch.long) for b in batch], dim=0)
    current_patches = torch.stack([torch.tensor(b[0][1], dtype=torch.long) for b in batch], dim=0)
    selected_tensors = []

    centroids = torch.stack([torch.tensor(b[1][0], dtype=torch.int) for b in batch], dim=0)

    if mode_box:
        selected_tensors.append(centroids)
    if mode_mask:
        selected_tensors.append(current_masks)
    if mode_patch:
        selected_tensors.append(current_patches)


    # Concatenate selected tensors along the last dimension
    combined_tensor = torch.cat(selected_tensors, dim=-1)

    # Reshape to add the singleton dimension
    combined_tensor = combined_tensor.unsqueeze(1)

    return combined_tensor, centroids
    
    

In [47]:
mode_box = True
mode_mask = False
mode_patch = False

dataloaders = {}
dataloaders['train'] = torch.utils.data.DataLoader(
    input_datasets['train'],
    batch_size=4,
    shuffle=True,
    num_workers=0,
    collate_fn=lambda batch: collate_fn(batch, mode_box, mode_mask, mode_patch)
)

dataloaders['test'] = torch.utils.data.DataLoader(
    input_datasets['test'],
    batch_size=4,
    shuffle=True,
    num_workers=0,
    collate_fn=lambda batch: collate_fn(batch, mode_box, mode_mask, mode_patch)
)

dataloaders['eval'] = torch.utils.data.DataLoader(
    input_datasets['eval'],
    batch_size=4,
    shuffle=True,
    num_workers=0,
    collate_fn=lambda batch: collate_fn(batch, mode_box, mode_mask, mode_patch)
)

In [48]:
for batch in dataloaders['train']:
    # input, output = batch
    # Process the batches as needed
    # print("Input Shape:", input.shape)
    # print("Output Shape:", output.shape)
    print("Input:", batch[0], "Centroids", batch[1])
    
    break  

cell mask  0  should be a number btw 0 and 67 
cell time shape and cell time (42,) [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1]
1000
array shape 0 42


IndexError: tuple index out of range

In [18]:
#create an LSTM RNN

#to create a pytorch LSTM 
#The first axis is the sequence itself
# the second indexes instances in the mini-batch
# third indexes elements of the input. 

# we would input T boxes
batch_size = 4

class LSTM(nn.Module):
    def __init__(self, T, hidden_dim, S, num_layers=2):
        #(input is )
        super(LSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
                           #input_size, hidden_size, num_layers
        self.lstm = nn.LSTM(T, hidden_dim, num_layers=2, batch_first=True) #stacking 2 LSTMs

        # The linear layer that maps from hidden state space to output space
        self.fc = nn.Linear(hidden_dim, S)
    def forward(self, input):


        #h_0: tensor of shape (D∗num_layers,N,Hout​) containing the initial hidden
        # state for each element in the input sequence. Defaults to zeros if (h_0, c_0) is not provided.
        #D = 2 if bidirectional is True, 1 otherwise
        #num layers is the number of layers
        #N = size of input
        #Hout is projection size
        h0 = torch.zeros(self.num_layers, input.size(0), self.hidden_dim).to(input.device)
        c0 = torch.zeros(self.num_layers, input.size(0), self.hidden_dim).to(input.device)

        # We need to detach as we are doing truncated backpropagation through time (BPTT)
        # If we don't, we'll backprop all the way to the start even after going through another batch
        out, _ = self.lstm(input, (h0.detach(), c0.detach()))

        # Decode the hidden state of the last time step
        out = self.fc(out)
        return out
        lstm_out, h_n = self.lstm(input, batch_size, -1) #output and final hidden state
        output = self.hidden2out(lstm_out.view(len(input), -1))
        #output = F.log_softmax(output, dim=1)
        return output

In [21]:
#mip only had one zplane 
# Example input data
#parameters for mask and patch
data_dim =  1000 #TO DO: find the biggest box and set it to this
learning_rate = 0.0001
epochs = 25

T = 10 #len predictions
hidden_dims = 100
output_dim = input_dim

def sigmoid(x):
    return 1/ 1+ np.exp(-x)

batch_size = 3
seq_length = 4
input_dim = 4
hidden_dim = 5
num_layers = 1
#input is (batch, seq, feature)
input_data = torch.randn(batch_size, input_dim, input_dim)

# Create LSTM model
lstm_model = LSTM(input_dim, hidden_dim, output_dim, num_layers)

# Forward pass with desired sequence length T
T = seq_length
output = lstm_model(input_data)

print("Input shape:", input_data.shape)
print("Output shape:", output.shape)


RuntimeError: Expected hidden[0] size (2, 3, 5), got [1, 3, 5]

In [ ]:

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

input_dim = 0
if mode_box == True:
    input_dim +=  4 #(T, 4) for the bounding boxes
if mode_mask == True:
    input_dim += box_shape.flatten #flattened version of [H, W]
if mode_patch == True:
    input_dim += box_shape.flatten #flattened version of [H, W]

learning_rate = 0.0001
epochs = 25
T = 2
hidden_dims = 100
output_dim = input_dim
batch_size = 1
model = LSTM(input_dim, hidden_dims, output_dim)
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

In [ ]:
# Do your training loop + eval, etc...
def train_model(model, criterion, optimizer, scheduler, num_epochs=epochs):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'dev']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:  #inputs shoudl be [B, S, D] #labes be a [B, T, D] batch, time, sequence shape
                inputs = inputs.to(device)

                 # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs, labels) #B, T, D

                    #LOSS AND PRED EDIT STUFF HERE AS WE TALKED ABOUT

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # deep copy the model
            if phase == 'dev' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())


    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model